In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
def read_owu_v1(file):
	root_path = 'dataset/datahow_2020/insilico_data'
	data = pd.read_excel(f'{root_path}/{file}.xlsx')
	col_names = ["run", "timesteps", "X:VCD", "X:Glc", "X:Gln", "X:NH4", "X:Lac", "X:Titer", "W:pH", "W:Temp", "F:Feed_Glc", "F:Feed_Gln",]
	owu_df = data.copy()
	owu_df.columns = col_names
	owu_df['time'] = (owu_df.timesteps / 24).astype(int)
	owu_df.set_index(['run', 'time'], inplace=True)
	return owu_df


def read_owu_v2(file):
	root_path = 'dataset/datahow_concise'
	data = pd.read_csv(f'{root_path}/{file}.csv')
	owu_df = data.copy()
	owu_df.set_index(['run', 'time'], inplace=True)
	return owu_df


def read_owu_v3(file, root_path = 'dataset/datahow_2022'):
	data = pd.read_csv(f'{root_path}/{file}.csv')
	owu_df = data.copy()
	owu_df.set_index(['run', 'time'], inplace=True)
	return owu_df

In [15]:
owu = read_owu_v3('owu', root_path='dataset/datahow_2022/interpolation/')
owu_test = read_owu_v3('owu_test', root_path='dataset/datahow_2022/interpolation/')
owu_ood = read_owu_v3('owu_ood')
owu_ood_titer = read_owu_v3('owu_ood_titer')

In [16]:
def plot_owu_data(owu_df, select_runs, height):
    max_cols_per_row = 5
    num_columns = len(owu_df.columns) - 1
    num_rows = (num_columns + max_cols_per_row - 1) // max_cols_per_row
    
    fig = make_subplots(
        rows=num_rows,
        cols=min(num_columns, max_cols_per_row),
        subplot_titles=owu_df.columns[1:]
    )
    
    for j in select_runs:
        plot_run_ix = owu_df.index.get_level_values("run") == j
        for i, c in enumerate(owu_df.columns):
            if i == 0:
                continue
            row = (i - 1) // max_cols_per_row + 1
            col = (i - 1) % max_cols_per_row + 1
            fig.add_trace(
                go.Scatter(
                    x=list(range(15)),
                    y=owu_df[c].values[plot_run_ix],
                    name="Run = " + str(j),
                    marker=dict(color=px.colors.qualitative.Plotly[j % 10]),
                ),
                row=row,
                col=col,
            )
    
    fig.update_layout(
        showlegend=False,
        title_text="Process variable evolution for selected runs",
        height=height,
    )
    fig.show()

# Plot Experiments

Here we visualise the generate experiments, especially ther evolution in time.

In [8]:
plot_owu_data(owu, select_runs=owu.index.get_level_values('run').unique().to_list()[:2], height=500)

In [9]:
def get_color(colorscale_name, loc):
	from _plotly_utils.basevalidators import ColorscaleValidator

	# first parameter: Name of the property being validated
	# second parameter: a string, doesn't really matter in our use case
	cv = ColorscaleValidator("colorscale", "")
	# colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...]
	colorscale = cv.validate_coerce(colorscale_name)

	if hasattr(loc, "__iter__"):
		return [get_continuous_color(colorscale, x) for x in loc]
	return get_continuous_color(colorscale, loc)

def get_continuous_color(colorscale, intermed):
	import plotly.colors
	from PIL import ImageColor

	"""
	Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
	color for any value in that range.
	"""
	if len(colorscale) < 1:
		raise ValueError("colorscale must have at least one color")

	hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

	if intermed <= 0 or len(colorscale) == 1:
		c = colorscale[0][1]
		return c if c[0] != "#" else hex_to_rgb(c)
	if intermed >= 1:
		c = colorscale[-1][1]
		return c if c[0] != "#" else hex_to_rgb(c)

	for cutoff, color in colorscale:
		if intermed > cutoff:
			low_cutoff, low_color = cutoff, color
		else:
			high_cutoff, high_color = cutoff, color
			break

	if (low_color[0] == "#") or (high_color[0] == "#"):
		# some color scale names (such as cividis) returns:
		# [[loc1, "hex1"], [loc2, "hex2"], ...]
		low_color = hex_to_rgb(low_color)
		high_color = hex_to_rgb(high_color)

	return plotly.colors.find_intermediate_color(
		lowcolor=low_color,
		highcolor=high_color,
		intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
		colortype="rgb",
	)

In [10]:
def plot_owu_data_color(owu, doe_df=None, highlight_run=0, select_color="Run_id", height=800):
    owu_df = owu.copy()
    if select_color == "Run_id":
        color_idx = np.repeat(np.array(list(range(len(doe_df)))), 15)
    if select_color == "Titer_14":
        color_idx = np.repeat(np.array(owu_df["X:Titer"][:, 14]), 15)
    if doe_df:
        if select_color in doe_df.columns:
            color_idx = np.repeat(np.array(doe_df[select_color]), 15)
    owu_df["color"] = color_idx

    owu_columns = owu_df.columns[1:-1]

    num_columns = len(owu_columns)
    max_cols_per_row = 5
    num_rows = (num_columns + max_cols_per_row - 1) // max_cols_per_row
    fig = make_subplots(
        rows=num_rows,
        cols=min(num_columns, max_cols_per_row), 
        subplot_titles=owu_columns)

    for i, c in enumerate(owu_columns):
        row = i // max_cols_per_row + 1
        col = i % max_cols_per_row + 1

        fig.add_trace(
            go.Scatter(
                x=list(range(15)),
                y=owu_df[c],
                mode="markers",
                marker=dict(
                    size=0,
                    color="rgba(0,0,0,0)",
                    colorscale="Portland",
                    cmin=min(color_idx),
                    cmax=max(color_idx),
                    colorbar=dict(thickness=40, title=str(select_color)),
                ),
                showlegend=False,
            ),
            row=row,
            col=col,
        )
        for color_val in np.unique(color_idx):
            color_val_norm = (color_val - min(color_idx)) / (
                max(color_idx) - min(color_idx)
            )
            owu_subset = owu_df[owu_df["color"] == color_val]
            fig.add_trace(
                go.Scatter(
                    x=owu_subset.index.get_level_values(1),
                    y=owu_subset[c],
                    mode="lines+markers",
                    name="Run id = "
                    + str(owu_df.index.get_level_values(0)[color_val == color_idx][0]),
                    marker=dict(color=get_color("Portland", color_val_norm)),
                ),
                row=row,
                col=col,
            )

    highlight_run_ix = owu_df.index.get_level_values("run") == highlight_run
    if doe_df:
        if highlight_run > len(doe_df):
            print("\n Highlighted run is higher than number of performed experiments \n")
    for i, c in enumerate(owu_columns):
        row = i // max_cols_per_row + 1
        col = i % max_cols_per_row + 1
        fig.add_trace(
            go.Scatter(
                x=list(range(15)),
                y=owu_df[c].values[highlight_run_ix],
                name="Run = " + str(highlight_run),
                marker=dict(color="black", size=10),
            ),
            row=row,
            col=col,
        )
    fig.update_layout(
        showlegend=False,
        title_text=f"Process variable evolution for selected runs and values colored by {select_color}",
        height=height
    )
    fig.show();

# Analyse all experiments

visualize all run from the generated dataset. By changing the `select_color` you decide on the coloring of the experiments. The options are:

* `Run_id` runs are colored by the order in which they appear in the dataset.
* `Titer_14` runs are colored by the amount of Titer at day 14 or the experiments.
* `glc_0` run are colored by the designed initial Glucose level
* `vcd_0` run are colored by the designed initial VCD level
* `feed_start` run are colored by the designed feeding start day
* `feed_end` run are colored by the designed feeding end day
* `feed_rate` run are colored by the designed Glucose feed rate

In [17]:
# DOE_FILENAME = 'dataset/datahow_concise/owu_doe.csv'
# doe = pd.read_csv(DOE_FILENAME)
# print(doe.columns)
plot_owu_data_color(owu, doe_df=None, highlight_run=0, select_color="Titer_14", height=500)

In [18]:
plot_owu_data_color(owu_test, doe_df=None, highlight_run=0, select_color="Titer_14", height=500)

In [19]:
plot_owu_data_color(owu_ood, doe_df=None, highlight_run=0, select_color="Titer_14", height=500)

In [20]:
plot_owu_data_color(owu_ood_titer, doe_df=None, highlight_run=0, select_color="Titer_14", height=500)